In [101]:
import sys
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../file_loader")
sys.path.append("../preprocessing-block")
sys.path.append("../features_calculation")


import ecg_file_loader
from ecg_file_loader import ECGFileLoader

import rpeaks_finder
from rpeaks_finder import *

import ectopic_beats_filtering
from ectopic_beats_filtering import *

import time_features
from time_features import *

reload(ecg_file_loader)
reload(rpeaks_finder)
reload(ectopic_beats_filtering)
reload(time_features)

<module 'time_features' from '/Users/pietrosaveri/Desktop/Pietro/•StartUps/Seizury/• src_blocks/check_codes/../features_calculation/time_features.py'>

In [2]:
loader = ECGFileLoader(use_s3=False)

In [3]:
processed_data = process_all_patients_rpeaks(loader, ['sub-001'])

INFO:rpeaks_finder:Processing R-peaks detection for 1 patients: ['sub-001']
INFO:rpeaks_finder:Loading data for patient sub-001
INFO:ecg_file_loader:Processing sub-001 run 01
INFO:ecg_file_loader:Processing sub-001 run 02
INFO:ecg_file_loader:Processing sub-001 run 03
INFO:ecg_file_loader:Processing sub-001 run 04
INFO:ecg_file_loader:Processing sub-001 run 05
INFO:ecg_file_loader:Processing sub-001 run 06
INFO:ecg_file_loader:Processing sub-001 run 07
INFO:ecg_file_loader:Processing sub-001 run 08
INFO:ecg_file_loader:Processing sub-001 run 09
INFO:rpeaks_finder:Processing sub-001 run 01
INFO:rpeaks_finder:ECG signal shape: (16721664,), sampling rate: 256.0 Hz
INFO:rpeaks_finder:Successfully processed sub-001 run 01: 72450 R-peaks detected
INFO:rpeaks_finder:Processing sub-001 run 02
INFO:rpeaks_finder:ECG signal shape: (5072896,), sampling rate: 256.0 Hz
INFO:rpeaks_finder:Successfully processed sub-001 run 02: 22104 R-peaks detected
INFO:rpeaks_finder:Processing sub-001 run 03
INFO:

In [4]:
processed_rr = process_dvc_for_session(processed_data['sub-001'][0], min_rr_s=0.3, max_rr_s=1.3)


--------------------------------------------------
Processing session 01 for patient sub-001
--------------------------------------------------
Initial state: 72449 RRs, 72450 timestamps.
--- Starting Ectopic Beat Filtering ---
Found 57 RR intervals > 1.3s. Deleting them to create gaps.
Found short RR interval (0.289s) at index 49661. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interval as an artifact.
Found short RR interval (0.293s) at index 31502. Analyzing merge options...
  > Right merge successful. New RR: 0.648s.
Found short RR interval (0.273s) at index 31486. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interval as an artifact.
Found short RR interval (0.270s) at index 16261. Analyzing merge options...
  > Right merge successful. New RR: 1.078s.
Found short RR interval (0.289s) at index 16227. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interva

In [93]:
processed_rr

{'patient_id': 'sub-001',
 'run_number': '01',
 'seizure_events': [],
 'annotations':      onset  duration eventType  lateralization  localization  vigilance  \
 0  14080.0      84.0      impd             NaN           NaN        NaN   
 1  23838.0       8.0      impd             NaN           NaN        NaN   
 
    confidence  channels             dateTime  recordingDuration  
 0         NaN       NaN  2000-01-01 00:00:00            65319.0  
 1         NaN       NaN  2000-01-01 00:00:00            65319.0  ,
 'ecg_signal': array([ 9.01747158e-06, -1.29744411e-05, -1.39778744e-05, ...,
         2.83817807e-08,  2.83817807e-08,  2.83817807e-08]),
 'sampling_rate': 256.0,
 'duration_seconds': 65319.0,
 'rpeaks': array([       9,      220,      432, ..., 16720440, 16720662, 16720877]),
 'rpeaks_times': array([3.51562500e-02, 8.59375000e-01, 1.68750000e+00, ...,
        6.53142188e+04, 6.53150859e+04, 6.53159258e+04]),
 'rr_intervals_ms': array([824.21875, 828.125  , 816.40625, ..., 890.

In [102]:
rr_peaks = processed_rr['dvc_rpeak_times_s']
fs = processed_rr['sampling_rate']

### Test with just one to see if it works

In [103]:
import pyhrv.time_domain as td

results = td.time_domain(rpeaks=rr_peaks, sampling_rate = fs, plot = False)

In [104]:
#non ci sono RR_var e SampEn
results

ReturnTuple(nni_counter=72506, nni_mean=900.3499434529556, nni_min=300.0, nni_max=1281.0, hr_mean=67.25518909607791, hr_min=46.8384074941452, hr_max=200.0, hr_std=6.852864039711369, nni_diff_mean=22.36743672850148, nni_diff_min=0, nni_diff_max=922, sdnn=83.10691062943927, sdnn_index=33.78894064083932, sdann=73.54570653825412, rmssd=34.37168079278846, sdsd=26.098268310812923, nn50=4793, pnn50=6.610578580787532, nn20=30511, pnn20=42.08123577684298, nni_histogram=None, tinn_n=0, tinn_m=0, tinn=0, tri_index=21.39451165535556)

### Test on sliding windows

In [105]:
features = compute_hrv_features(rr_peaks, sampling_rate=fs, window_minutes=5)

for i, window in enumerate(features):
    start = window['start_time_s']
    end = window['end_time_s']
    feats = window['features']
    print(f"\nWindow {i+1}: {start:.2f}s → {end:.2f}s")
    print(feats)

testing
Processing window


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")



Window 1: 0.00s → 300.15s
ReturnTuple(nni_counter=362, nni_mean=828.646408839779, nni_min=808.0, nni_max=855.0, hr_mean=72.41680340334962, hr_min=70.17543859649123, hr_max=74.25742574257426, hr_std=0.8321432655419995, nni_diff_mean=9.767313019390581, nni_diff_min=0, nni_diff_max=31, sdnn=9.549802441937342, sdnn_index=nan, sdann=nan, rmssd=11.962313860979709, sdsd=6.915854228114157, nn50=0, pnn50=0.0, nn20=28, pnn20=7.756232686980609, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.290909090909091)

Window 2: 300.15s → 599.76s
ReturnTuple(nni_counter=356, nni_mean=838.6741573033707, nni_min=796.0, nni_max=875.0, hr_mean=71.55628633066264, hr_min=68.57142857142857, hr_max=75.37688442211055, hr_std=1.0300597696191698, nni_diff_mean=10.416901408450704, nni_diff_min=0, nni_diff_max=47, sdnn=12.08306954835555, sdnn_index=nan, sdann=nan, rmssd=12.616778455356554, sdsd=7.12842218173045, nn50=0, pnn50=0.0, nn20=17, pnn20=4.788732394366197, nni_histogram=None, tin

In [106]:
features[0]['features'] #prima window

ReturnTuple(nni_counter=362, nni_mean=828.646408839779, nni_min=808.0, nni_max=855.0, hr_mean=72.41680340334962, hr_min=70.17543859649123, hr_max=74.25742574257426, hr_std=0.8321432655419995, nni_diff_mean=9.767313019390581, nni_diff_min=0, nni_diff_max=31, sdnn=9.549802441937342, sdnn_index=nan, sdann=nan, rmssd=11.962313860979709, sdsd=6.915854228114157, nn50=0, pnn50=0.0, nn20=28, pnn20=7.756232686980609, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.290909090909091)

### Test on 1 minute "pakages"

In [ ]:
# CI METTE UN SACCO

features_sliding = compute_hrv_features_sliding(
    rr_peaks,
    sampling_rate=fs,
    window_minutes=5,
    step_minutes=1
)

for f in features_sliding[:5]:
    print(f"\nMinute {f['minute']:.0f}: {f['start_time_s']:.2f}s → {f['end_time_s']:.2f}s")
    print(f['features'])


In [111]:
# SO CHE PROBABILEMTE ROMBO UN RR-INTERAL MA ERA SOLO PER PROVARE SE IL WINDOWING ERA GIUSTO

rr_rr_peaks_short = rr_peaks[:5000]

In [113]:
features_sliding = compute_hrv_features_sliding(
    rr_rr_peaks_short,
    sampling_rate=fs,
    window_minutes=5,
    step_minutes=1
)

for f in features_sliding[:5]:
    print(f"\nMinute {f['minute']:.0f}: {f['win_start_time_s']:.2f}s → {f['win_end_time_s']:.2f}s")
    print(f['features'])


Starting sliding HRV computation...
Finished processing.

Minute 5: 0.04s → 300.15s
ReturnTuple(nni_counter=361, nni_mean=828.6592797783934, nni_min=808.0, nni_max=855.0, hr_mean=72.41569888651517, hr_min=70.17543859649123, hr_max=74.25742574257426, hr_std=0.8330324542094124, nni_diff_mean=9.783333333333333, nni_diff_min=0, nni_diff_max=31, sdnn=9.559912164395186, sdnn_index=nan, sdann=nan, rmssd=11.977061409210526, sdsd=6.9187686282658944, nn50=0, pnn50=0.0, nn20=28, pnn20=7.777777777777778, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.311926605504587)

Minute 6: 60.30s → 360.33s
ReturnTuple(nni_counter=360, nni_mean=830.7, nni_min=808.0, nni_max=867.0, hr_mean=72.23871500660283, hr_min=69.20415224913495, hr_max=74.25742574257426, hr_std=0.8695983609837146, nni_diff_mean=9.746518105849582, nni_diff_min=0, nni_diff_max=32, sdnn=10.034425424174259, sdnn_index=nan, sdann=nan, rmssd=11.99315031991596, sdsd=6.9983903072655, nn50=0, pnn50=0.0, nn20=27, pnn2

In [114]:
features_sliding[0] #prima window

{'minute': 5.002473958333334,
 'win_start_time_s': 0.03515625,
 'win_end_time_s': 300.1484375,
 'features': ReturnTuple(nni_counter=361, nni_mean=828.6592797783934, nni_min=808.0, nni_max=855.0, hr_mean=72.41569888651517, hr_min=70.17543859649123, hr_max=74.25742574257426, hr_std=0.8330324542094124, nni_diff_mean=9.783333333333333, nni_diff_min=0, nni_diff_max=31, sdnn=9.559912164395186, sdnn_index=nan, sdann=nan, rmssd=11.977061409210526, sdsd=6.9187686282658944, nn50=0, pnn50=0.0, nn20=28, pnn20=7.777777777777778, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.311926605504587)}